In [2]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


# keras 튜너

## keras 튜너 빌드용

In [3]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    # hp 객체에서 하이퍼 파라미터 값을 샘플링
    # 샘플링한 값은("units")는 파이썬 상수임
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    # 여러 종류의 하이퍼파라미터를 제공
    # Int, Float, Boolean, Choice등이 있다
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

In [4]:
# keras tuner의 HyperModel을 상속받은 튜너를

import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

## [케라스 튜너 종류](https://keras.io/api/keras_tuner/tuners/)

In [5]:
# KerasTuner의 내장 튜너
# BayesianOptimization 튜너는 이전 선택ㄱ의 결과를 바탕으로 최상의 하이퍼파라미터 값을 스마트하게 예측하게해줌
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    # 코랩에서 정상 실행만 확인하기 위해 최대 탐색 횟수를 100에서 10으로 줄입니다
    max_trials=10, # 100
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


## Mnist 데이터셋 load 및 BayesianOptimization 으로 하이퍼파라미터 찾기

In [7]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]

num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]

tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=100,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 10 Complete [00h 00m 53s]
val_accuracy: 0.9748499989509583

Best val_accuracy So Far: 0.9760499894618988
Total elapsed time: 00h 11m 38s


In [10]:
# best 파라미터 확인
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [11]:
print(best_hps)
print("-"*30)
print(best_hps[0].Choice(name="optimizer", values=["rmsprop", "adam"]))

# best 파라미터로 model build
model = build_model(best_hps[0])

[<keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7ee9944199c0>, <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7ee99c119240>, <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7ee9946e7f10>, <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7ee994f412d0>]
------------------------------
rmsprop


In [14]:
from keras_tuner.engine.hyperparameters import Int
print(1)
print(best_hps[0]._conditions_are_active(Int(name="units", min_value=0, max_value=1).conditions))
print("-"*30)

print(2)
print(best_hps[0]._hps)
print("-"*30)

print(3)
print(best_hps[0]._exists('units', []))
print("-"*30)

print(4)
print(best_hps[0].values)
print("-"*30)

print(5)
print(best_hps[0].Int(name="units", min_value=0, max_value=1))
print("-"*30)

print(6)
model.get_config()


1
True
------------------------------
2
defaultdict(<class 'list'>, {'units': [Int(name: 'units', min_value: 16, max_value: 64, step: 16, sampling: linear, default: 16)], 'optimizer': [Choice(name: 'optimizer', values: ['rmsprop', 'adam'], ordered: False, default: rmsprop)]})
------------------------------
3
True
------------------------------
4
{'units': 64, 'optimizer': 'rmsprop'}
------------------------------
5
64
------------------------------
6


{'name': 'sequential_1',
 'layers': [{'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense_2',
    'trainable': True,
    'dtype': 'float32',
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'GlorotUniform',
     'config': {'seed': None},
     'registered_name': None},
    'bias_initializer': {'module': 'keras.initializers',
     'class_name': 'Zeros',
     'config': {},
     'registered_name': None},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'registered_name': None},
  {'module': 'keras.layers',
   'class_name': 'Dense',
   'config': {'name': 'dense_3',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'softmax',
    'use_bias': True,
    'kernel_initializer': {'module': 'keras.initializers',
     'class_name': 'Gloro

In [15]:
# 최상의 epoch 횟수 찾기
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [17]:
# 위 훈련 데이터셋으로 찾은 최상의 epoch보다 20%만큼 더 많이 epoch을 지정하여 다시 훈련
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

In [18]:
# 모델 훈련 및 평가
best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100
391/391 [==============================] - 2s 4ms/step - loss: 0.4213 - accuracy: 0.8863 - val_loss: 0.2259 - val_accuracy: 0.9366
Epoch 2/100
391/391 [==============================] - 2s 4ms/step - loss: 0.2183 - accuracy: 0.9378 - val_loss: 0.1820 - val_accuracy: 0.9509
Epoch 3/100
391/391 [==============================] - 2s 5ms/step - loss: 0.1711 - accuracy: 0.9516 - val_loss: 0.1537 - val_accuracy: 0.9571
Epoch 4/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1412 - accuracy: 0.9603 - val_loss: 0.1345 - val_accuracy: 0.9638
Epoch 5/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1194 - accuracy: 0.9664 - val_loss: 0.1239 - val_accuracy: 0.9652
Epoch 6/100
391/391 [==============================] - 1s 3ms/step - loss: 0.1035 - accuracy: 0.9709 - val_loss: 0.1159 - val_accuracy: 0.9665
Epoch 7/100
391/391 [==============================] - 1s 3ms/step - loss: 0.0911 - accuracy: 0.9749 - val_loss: 0.1124 - val_accuracy: 0.9690

In [19]:
best_models = tuner.get_best_models(top_n)

In [20]:
print(best_models)

[<keras.src.engine.sequential.Sequential object at 0x7ee994158ac0>, <keras.src.engine.sequential.Sequential object at 0x7ee99415aa70>, <keras.src.engine.sequential.Sequential object at 0x7ee98589b130>, <keras.src.engine.sequential.Sequential object at 0x7ee985727d00>]
